#  Data Cleaning

Some mistakes were made during the scraping process due to the variation in data format. Some were caught early and fixed for subsequent scraping tasks.

The two types of mistakes are:
1. Country 
2. Date

The country column was expected to be a single word. However, United States was 'the United States'. I changed the data so that 'the' became 'US' in the country column.

The date value is in a human-readable format. For example, May 23, 2020 was used instead of 05-23-2020. This is fixed with pandas' to_datetime function.

Since 'country' and 'date' were scraped from the same sentence, samples that had 'the United States' also had two extra characters attached to the date, which threw an error when using the to_datetime function. The data was split, sliced, and rejoined together before the to_datetime function.

## Imports

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv

In [14]:
df = pd.read_csv('data/raw.csv')

In [15]:
df.head()

,0,1,2,3,4
0,3,Pricy for it’s quality,Giving four stars because the inner layer is j...,Canada,"June 4, 2020"
1,3,They fit small,"The fabric, elastic & overall quality is great...",Canada,"June 27, 2020"
2,4,Needs filters that aren't included!,Most of the reason I ordered this is because I...,Canada,"June 11, 2020"
3,4,fast delivery & comfortable,i recieved the wrong colour but it saves me fr...,Canada,"May 20, 2020"
4,4,Comfortable with customizable adjustment,I bought 4 of these and will possibly purchase...,Canada,"June 23, 2020"


In [16]:
df.iloc[:,3].value_counts()

the          5528
Canada       3614
France         62
Italy          49
Germany        37
Japan           6
Australia       2
Mexico          1
Name: 3, dtype: int64

In [19]:
# replacing 'the' with 'US' in the location column
df.loc[df['3'] == 'the','3'] = 'US'

In [20]:
df.tail()

,0,1,2,3,4
9294,2,Just okay.,Poor stitching work on both of the mask ordered.,US,"May 23, 2020"
9295,2,Disappointed,Waste of money,US,"July 2, 2020"
9296,2,Improvement required as ear loop should be adj...,Dislike.,US,"July 22, 2020"
9297,2,"Nasenclip nicht entfernbar, trotz Angabe auf V...",Also generell wäre es eine tolle Maske. Aber i...,Germany,"July 31, 2020"
9298,2,Schlechter als jede Stoffmaske,Das beste zuerst. Es ist eine Maske. Für den P...,Germany,"October 4, 2020"


In [21]:
df.iloc[:,3].value_counts()

US           5528
Canada       3614
France         62
Italy          49
Germany        37
Japan           6
Australia       2
Mexico          1
Name: 3, dtype: int64

In [23]:
df['4'] = df['4'].apply(lambda x: " ".join(x.split()[-3:]))

In [24]:
df.iloc[:,4].value_counts()

December 6, 2020     73
August 12, 2020      69
August 9, 2020       68
September 4, 2020    67
December 4, 2020     63
                     ..
January 30, 2020      1
May 29, 2020          1
January 7, 2020       1
May 23, 2020          1
May 20, 2020          1
Name: 4, Length: 262, dtype: int64

In [25]:
df['4'] = pd.to_datetime(df['4'])

In [26]:
df.iloc[:,4].value_counts()

2020-12-06    73
2020-08-12    69
2020-08-09    68
2020-09-04    67
2020-12-04    63
              ..
2020-02-25     1
2020-05-31     1
2020-05-26     1
2020-01-07     1
2020-05-17     1
Name: 4, Length: 262, dtype: int64

## Missing Values

First, I checked the number of null values in each column. There are only a handful of NaN values in the review title and review text columns. Rating, location, and date do not contain null values, as expected since a rating is required and location and time are stamped automatically.

The simplest way was to remove them altogether, but I noticed some columns contained valid reviews. Therefore, I removed only the rows with both empty review titles and review texts (i.e. only a star rating is given). There is one sample where the customer wrote 'n/a' in the review text and I removed that as well. For the rest, I replaced NaN with an empty string.

In [27]:
df.isnull().sum()

0    0
1    6
2    4
3    0
4    0
dtype: int64

In [28]:
df[df['1'].isnull()]

,0,1,2,3,4
2643,5,NaN,"It's quite comfortable, however, the loop stri...",Canada,2021-02-01
2694,5,NaN,NaN,Canada,2020-10-04
3647,1,NaN,NaN,Canada,2021-01-13
6209,4,NaN,N/a,US,2020-07-22
6792,3,NaN,"Dislike, I use for work but too small for me",Canada,2020-10-16
7711,1,NaN,NaN,Canada,2021-01-13


In [29]:
df[df['2'].isnull()]

,0,1,2,3,4
2694,5,NaN,NaN,Canada,2020-10-04
2797,5,Convenient,NaN,Canada,2020-08-11
3647,1,NaN,NaN,Canada,2021-01-13
7711,1,NaN,NaN,Canada,2021-01-13


In [30]:
df.dropna(subset=['1','2'], how='all', inplace=True) 

In [31]:
df[df['1'].isnull()]

,0,1,2,3,4
2643,5,NaN,"It's quite comfortable, however, the loop stri...",Canada,2021-02-01
6209,4,NaN,N/a,US,2020-07-22
6792,3,NaN,"Dislike, I use for work but too small for me",Canada,2020-10-16


In [32]:
df[df['2'].isnull()]

,0,1,2,3,4
2797,5,Convenient,NaN,Canada,2020-08-11


In [33]:
df.drop(6209, inplace = True)

In [34]:
df[df['1'].isnull()]

,0,1,2,3,4
2643,5,NaN,"It's quite comfortable, however, the loop stri...",Canada,2021-02-01
6792,3,NaN,"Dislike, I use for work but too small for me",Canada,2020-10-16


In [35]:
df.replace(np.nan, '' , inplace=True)
# df.fillna('', inplace=True) works as well

In [36]:
df.isnull().sum()

0    0
1    0
2    0
3    0
4    0
dtype: int64

## Rename the Columns

In [37]:
df = df.rename(columns={'0': 'rating', \
                        '1':'title', \
                        '2':'review', \
                        '3': 'country', \
                        '4': 'date'
                       })


In [38]:
df.head()

,rating,title,review,country,date
0,3,Pricy for it’s quality,Giving four stars because the inner layer is j...,Canada,2020-06-04
1,3,They fit small,"The fabric, elastic & overall quality is great...",Canada,2020-06-27
2,4,Needs filters that aren't included!,Most of the reason I ordered this is because I...,Canada,2020-06-11
3,4,fast delivery & comfortable,i recieved the wrong colour but it saves me fr...,Canada,2020-05-20
4,4,Comfortable with customizable adjustment,I bought 4 of these and will possibly purchase...,Canada,2020-06-23


# Saving the Cleaned Data

In [39]:
df.to_csv(r'data/cleaned.csv', index=False)